In [451]:
import networkx as nx
import numpy as np
import cv2
import json
import networkx as nx
import re
import argparse
import glob
import os

from route_generator import iter_all_white_points, draw_waypoint, rdp_algorithm, recorded_find_path
from add_path_to_graph import add_path, read_adjlist_with_tuples

##################### Input Town number #####################

town = int(input("Enter town number: 1, 2, 3, 7, or 10"))

if town == 10:
    waypoint_map = cv2.imread("./waypoint_maps/Town10HD.png", 1)
    town_name = "Town10HD"
else:
    town_name = f"Town0{town}"
    waypoint_map = cv2.imread(f"./waypoint_maps/{town_name}.png", 1)


##################### Read latest graph list of the given town #####################

list_of_files = glob.glob(f'graph_list/{town_name}/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)

if town == 10:
    index = int(latest_file[30:-8])
    new_file_name = latest_file[:30] + str(index+1) + latest_file[-8:]
else:
    index = int(latest_file[27:-8])
    new_file_name = latest_file[:27] + str(index+1) + latest_file[-8:]

G = read_adjlist_with_tuples(latest_file)

print(latest_file)

graph_list/Town03\3_graph_v105.adjlist


In [452]:
################# Change start, end point #################

reverse = input("Do reverse search or not? (y for reverse search, other for original search)\n")

start, end =(2740, 1212), (3098, 838) 
if reverse == "y":
    start, end = end, start


In [453]:
if start == (0,0) or end == (0,0):
    print("Didn't change to your point!!!!!!!!")

all_whites_pos = iter_all_white_points(waypoint_map)

new_img = waypoint_map.copy()
new_img[start[0]][start[1]] = np.array([0, 0, 255])
new_img[end[0]][end[1]] = np.array([0, 0, 255])

cv2.circle(new_img, start[::-1], radius=5, color=(255, 0, 0), thickness=5)
cv2.circle(new_img, end[::-1], radius=5, color=(0, 255, 0), thickness=5)


print("Start point:", start)
print("End point:", end)
print("Finding Path...")
# final_path = find_path(start, end, all_whites_pos)
flag, final_path = recorded_find_path(start, end, all_whites_pos, "", False)

##### Path not found #####
if flag == False:
    print("Not found!!!")
    print(final_path)
    for i, j in final_path[0]:
        new_img[i][j] = np.array([255, 255, 0])
    
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/not_found/test__{start}_{end}.png", new_img)
else:
    print("found!!!")
    print(final_path[0])
    print("Adding to the graph list...")
    
    if reverse == "y":
        fp = final_path[0][::-1]
        fp = [fp]
        add_path(G, fp, new_file_name)
    else:
        add_path(G, final_path, new_file_name)
    
    print(new_file_name, "Store!")
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/found/test__{start}_{end}.png", rdp_img)

Start point: (2740, 1212)
End point: (3098, 838)
Finding Path...


c:\Users\User\Desktop\build-CARLA-graph\route_generator.py:193: RuntimeWarning: divide by zero encountered in longlong_scalars
  win_m = (p[-2][1] - tp2[1]) / (p[-2][0] - tp2[0])


Get!!!
found!!!
[(2740, 1212), (2743, 1208), (2746, 1204), (2749, 1200), (2751, 1197), (2753, 1193), (2756, 1190), (2760, 1185), (2763, 1180), (2765, 1178), (2767, 1174), (2770, 1170), (2773, 1167), (2776, 1163), (2778, 1160), (2780, 1156), (2782, 1150), (2783, 1149), (2785, 1147), (2786, 1146), (2788, 1144), (2789, 1143), (2791, 1141), (2794, 1138), (2797, 1135), (2799, 1133), (2800, 1132), (2802, 1130), (2803, 1129), (2805, 1127), (2806, 1126), (2808, 1124), (2811, 1121), (2814, 1118), (2816, 1116), (2817, 1115), (2819, 1113), (2820, 1112), (2822, 1110), (2826, 1107), (2828, 1105), (2829, 1104), (2831, 1102), (2834, 1098), (2836, 1096), (2839, 1093), (2843, 1090), (2845, 1088), (2846, 1087), (2848, 1085), (2851, 1082), (2854, 1079), (2856, 1077), (2857, 1076), (2859, 1074), (2860, 1073), (2862, 1071), (2863, 1070), (2865, 1068), (2866, 1067), (2868, 1065), (2870, 1063), (2873, 1060), (2876, 1057), (2879, 1054), (2882, 1051), (2885, 1048), (2888, 1046), (2890, 1043), (2893, 1040), (28

In [454]:
############### Quick test for other start point with the same goal point
start, end = (2825, 1336), (3098, 838)

fp = nx.shortest_path(G,  start,  end)
draw_img = draw_waypoint(waypoint_map, start, end, fp)
rdp_img, _ = rdp_algorithm(draw_img, fp)
cv2.imwrite(f"test/check__{start}_{end}.png", rdp_img)
print(fp)

[(2825, 1336), (2820, 1336), (2809, 1336), (2797, 1336), (2784, 1336), (2778, 1335), (2760, 1336), (2750, 1330), (2738, 1325), (2728, 1320), (2719, 1312), (2708, 1302), (2710, 1297), (2709, 1288), (2707, 1284), (2719, 1277), (2730, 1270), (2744, 1263), (2738, 1257), (2723, 1251), (2731, 1244), (2740, 1236), (2748, 1224), (2736, 1219), (2740, 1212), (2743, 1208), (2751, 1197), (2760, 1185), (2767, 1174), (2776, 1163), (2782, 1150), (2785, 1147), (2794, 1138), (2802, 1130), (2811, 1121), (2819, 1113), (2828, 1105), (2837, 1096), (2845, 1088), (2854, 1079), (2862, 1071), (2873, 1060), (2882, 1051), (2890, 1043), (2899, 1034), (2907, 1026), (2916, 1018), (2924, 1009), (2933, 1001), (2942, 992), (2950, 984), (2959, 975), (2967, 967), (2976, 958), (2984, 950), (2993, 942), (3001, 933), (3010, 925), (3018, 916), (3027, 908), (3035, 899), (3044, 891), (3052, 883), (3061, 874), (3070, 866), (3078, 857), (3087, 849), (3095, 840), (3098, 838)]
